### Retrieval Augmented Generation with ollama and chromadb

#### 1. Preliminaries

In [ ]:
import gradio as gr
import ollama
import chromadb
import html

# on Docker image:
chromaclient = chromadb.PersistentClient(path="/home/imagery/crc5rag")
collection = chromaclient.get_collection(name="crc5rag")

# on my machine:
#chromaclient = chromadb.PersistentClient(path="/home/mort/crc5imagery/crc5rag")
#collection = chromaclient.get_collection(name="crc5rag")

#### 2. Choose a distilled LLM model appropriate to your local computer or one of the cloud versions. 

In [ ]:
#model = 'deepseek-r1:7b'
#model = 'deepseek-r1:14b'
#model = 'deepseek-v3.1:671b-cloud'
model = 'mistral-large-3:675b-cloud'

If you have chosen a cloud version above, you require a (free) ollama account. Open a terminal window in the Launcher and enter 

    ollama serve &

Then, in a new terinal window, enter

    ollama signin

and follow the instructions.  

#### 3. Pull the model and the nomic text embedder 
This will take a few minutes the first time. Note that Ollama uses a docker-like syntax to pull LLM's.

In [ ]:
%%capture
!ollama pull $model
!ollama pull nomic-embed-text

#### 4. Execute RAG queries on the textbook contents

In [ ]:
def ragask(query):
    # Embed the query
    queryembed = ollama.embed(model="nomic-embed-text", input=query)['embeddings']
    # Retrieve related documents (eight 512-token chunks)
    relateddocs = '\n\n'.join(collection.query(query_embeddings=queryembed, n_results=8)['documents'][0])
    # Generate a response
    prompt = f"Answer the question: {query}, referring to the following text as a resource: {relateddocs}"
    response = ollama.generate(model=model, prompt=prompt, stream=False)['response']   
    # Ensure the response is valid Markdown
    return html.escape(response)

# Launch Gradio Interface
# If you want to re-start this cell, bump the server port by one (e.g. ... .launch(server_name="0.0.0.0", server_port=7861)
gr.Interface(fn=ragask,inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
    outputs="markdown",
    description="Ask questions about the book contents",
    title="Image Analysis, Classification and Change Detection in Remote Sensing").launch(server_name="0.0.0.0", server_port=7860)